# ResNet for tabular data

In [1]:
from comet_ml import Experiment

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from tqdm.notebook import tqdm_notebook as tqdm

from matplotlib import pyplot as plt
import seaborn as sns

COMET_ML_API_KEY = ""
COMET_ML_PROJECT_NAME = "resnet-tabular"
COMET_ML_WORKSPACE = "nikolai-semenov"

## Data preparation

Sources:
1. https://archive.ics.uci.edu/ml/datasets/SUSY
1. https://archive.ics.uci.edu/ml/datasets/HIGGS

In [2]:
%%time
SUSY = pd.read_csv('data/SUSY.csv.gz', header=None)
HIGGS = pd.read_csv('data/HIGGS.csv.gz', header=None)

CPU times: user 2min 52s, sys: 4.02 s, total: 2min 56s
Wall time: 2min 57s


## Experiment preparation

View panel: https://www.comet.ml/nikolai-semenov/resnet-tabular/view/Iy69366eJvLEKY4yKNkqenI1j

In [3]:
def perform_experiment(
    model,
    X,
    y,
    test_size,
    batch_size,
    num_epochs,
    hyper_params={},
    log_every=100,
    random_state=42
):
    experiment = Experiment(
        api_key=COMET_ML_API_KEY,
        project_name=COMET_ML_PROJECT_NAME,
        workspace=COMET_ML_WORKSPACE,
    )
    if hyper_params:
        experiment.log_parameters(hyper_params)
    experiment.log_parameters({
        "test_size": test_size,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
    })
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    X_train = torch.Tensor(X_train)
    X_test = torch.Tensor(X_test)
    y_train = torch.Tensor(y_train)
    y_test = torch.Tensor(y_test)
    
    train_dataset = TensorDataset(X_train, y_train)    
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999))


    step = 0
    for epoch in tqdm(range(num_epochs), desc="Epoch", total=num_epochs):
        with experiment.train():
            model.train()
            for i, (x, y_true) in enumerate(train_loader):
                optimizer.zero_grad()
                y = model(x)
                loss = criterion(y, y_true)
                loss.backward()
                optimizer.step()
                
                step += 1  
                if step % log_every == 0:
                    experiment.log_metric("batch_loss", loss.item(), step=step, epoch=epoch)
            model.eval()
            with torch.no_grad():
                train_score = roc_auc_score(y_train.numpy(), model(X_train).numpy())
                experiment.log_metric("score", train_score, step=epoch, epoch=epoch)
        with experiment.test():
            model.eval()
            with torch.no_grad():
                test_score = roc_auc_score(y_test.numpy(), model(X_test).numpy())
                experiment.log_metric("score", test_score, step=epoch, epoch=epoch)
    experiment.end()

## Baseline

In [4]:
class BaselineModel(nn.Module):
    def __init__(self, input_size, n):
        super().__init__()
        self.baseline_model = nn.Sequential(
            nn.Linear(input_size, 8 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(8 * n, 6 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(6 * n, 4 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(4 * n, 3 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(3 * n, 2 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(2 * n, 1),
            nn.Sigmoid()
        )
        
        def init_weights(layer):
            if type(layer) == nn.Linear:
                nn.init.normal_(layer.weight, std=1e-3)
        
        self.baseline_model.apply(init_weights)

    def forward(self, x):
        return self.baseline_model(x).reshape(-1)

In [12]:
for test_size in [0.99, 0.9]:
    for n in [1, 4, 16]:
        perform_experiment(
            model=BaselineModel(SUSY.shape[1] - 1, n),
            X=SUSY.values[:,1:],
            y=SUSY.values[:,0],
            test_size=test_size,
            batch_size=2048,
            num_epochs=100,
            hyper_params={
                "model": "baseline",
                "dataset": "SUSY",
                "n": n,
            },
            log_every=int((1 - test_size) * 200),
            random_state=42)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/e1b1cf45f6be4011be63cc764fc1f33a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [67]           : (0.5, 0.871193763455273)
COMET INFO:     train_batch_ROC_AUC [850] : (0.5, 0.8992672450096859)
COMET INFO:     train_batch_loss [850]    : (0.39205673336982727, 0.6951355338096619)
COMET INFO:     train_loss [171]          : (0.39205673336982727, 0.6905364990234375)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 4
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET I

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/7a5bbe7cc2d24768be83c81604e666cb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.5, 0.8690514289959075)
COMET INFO:     train_batch_loss [1250] : (0.4085255265235901, 0.6944254636764526)
COMET INFO:     train_loss [260]        : (0.41125988960266113, 0.6930562257766724)
COMET INFO:     train_score [100]       : (0.5, 0.868340939809526)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 1
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (12 KB)
COMET INFO:     environment details : 1
COMET IN

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/570e7c2403ab4999993630b362bc047e



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/570e7c2403ab4999993630b362bc047e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.5256859208249698, 0.8720553175609976)
COMET INFO:     train_batch_loss [1250] : (0.4023769497871399, 0.6971518993377686)
COMET INFO:     train_loss [260]        : (0.4042855501174927, 0.6971518993377686)
COMET INFO:     train_score [100]       : (0.5266600421445448, 0.8730356614242216)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 4
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (12 KB)
COMET INFO:     en

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/08c664cd1b80499e99794957ae1d61a9



...

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/ef24423a406e41c3b386ec3090e9f322



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/ef24423a406e41c3b386ec3090e9f322
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.8527110534664829, 0.8753595417888297)
COMET INFO:     train_batch_loss [1289] : (0.39231160283088684, 0.7072004675865173)
COMET INFO:     train_loss [2473]       : (0.35656487941741943, 0.7015822529792786)
COMET INFO:     train_score [100]       : (0.8522012092393245, 0.875217558717703)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 4
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.9
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (12 KB)
COMET INFO:     en

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/55ecbb54c7ac4a6fa4a54689ade95992



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/55ecbb54c7ac4a6fa4a54689ade95992
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.8676712479038908, 0.8764915101682792)
COMET INFO:     train_batch_loss [1289] : (0.3933541774749756, 0.70271235704422)
COMET INFO:     train_loss [2473]       : (0.35201990604400635, 0.7065621018409729)
COMET INFO:     train_score [100]       : (0.8669887320855292, 0.8789654588527495)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 16
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.9
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (12 KB)
COMET INFO:     env

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


In [13]:
for test_size in [0.99, 0.9]:
    for n in [1, 4, 16]:
        perform_experiment(
            model=BaselineModel(HIGGS.shape[1] - 1, n),
            X=HIGGS.values[:,1:],
            y=HIGGS.values[:,0],
            test_size=test_size,
            batch_size=2048,
            num_epochs=100,
            hyper_params={
                "model": "baseline",
                "dataset": "HIGGS",
                "n": n,
            },
            log_every=int((1 - test_size) * 200),
            random_state=42)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/c5a1df493d334e77a2e68f6ada0f8228



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/c5a1df493d334e77a2e68f6ada0f8228
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.4999997196827467, 0.668143728176235)
COMET INFO:     train_batch_loss [2700] : (0.6272420883178711, 0.6962411403656006)
COMET INFO:     train_loss [540]        : (0.6298243403434753, 0.6951794624328613)
COMET INFO:     train_score [100]       : (0.5, 0.6692423597100982)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 1
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (13 KB)
COMET INFO:     environment detai

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/f9e0aca8d35d478faebf711d07efb7fc



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/f9e0aca8d35d478faebf711d07efb7fc
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.4654738961236336, 0.7788292816354289)
COMET INFO:     train_batch_loss [2700] : (0.5260930061340332, 0.7238398194313049)
COMET INFO:     train_loss [540]        : (0.5391815304756165, 0.7238398194313049)
COMET INFO:     train_score [100]       : (0.46960970298446314, 0.7862313828716758)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 4
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (13 KB)
COMET INFO:     

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/57235f6de6844a4384c9ddf2ef2bd05a



...

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/ccb5b6abb7504596a4e7fc7fe31e90ce



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/ccb5b6abb7504596a4e7fc7fe31e90ce
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.6758191970245504, 0.8264466001232591)
COMET INFO:     train_batch_loss [2831] : (0.47402480244636536, 0.69239342212677)
COMET INFO:     train_loss [5385]       : (0.47207799553871155, 0.6938610076904297)
COMET INFO:     train_score [100]       : (0.6759172097302438, 0.8274369674951643)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 4
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.9
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (13 KB)
COMET INFO:     en

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/3ffc0b7eda264c86a591a0a56424a9ec



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/3ffc0b7eda264c86a591a0a56424a9ec
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.7315781731276296, 0.8465206299455973)
COMET INFO:     train_batch_loss [2831] : (0.43582800030708313, 0.6931089758872986)
COMET INFO:     train_loss [5385]       : (0.4083406329154968, 0.6947827935218811)
COMET INFO:     train_score [100]       : (0.7314922409914278, 0.8602162494947416)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 16
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.9
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (13 KB)
COMET INFO:     

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


In [6]:
perform_experiment(
    model=BaselineModel(SUSY.shape[1] - 1, 8),
    X=SUSY.values[:,1:],
    y=SUSY.values[:,0],
    test_size=0.99,
    batch_size=2048,
    num_epochs=100,
    hyper_params={
        "model": "baseline",
        "dataset": "SUSY",
        "n": 8,
    },
    log_every=2,
    random_state=42)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/927492876e424d8ab57bc129b1e95cca
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [62]        : (0.7973706603440791, 0.871258547126281)
COMET INFO:     train_batch_loss [787] : (0.40036675333976746, 0.7019533514976501)
COMET INFO:     train_loss [164]       : (0.4091244041919708, 0.701042652130127)
COMET INFO:     train_score [63]       : (0.7958325106741773, 0.8709467166643625)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 64
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename      

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/aabe81bc63734b39b361528b6f920aa0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.796108365206421, 0.8714959817245549)
COMET INFO:     train_batch_loss [1250] : (0.3917686939239502, 0.6926658153533936)
COMET INFO:     train_loss [260]        : (0.3917686939239502, 0.6929274797439575)
COMET INFO:     train_score [100]       : (0.7939503975286111, 0.8716572365188996)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 8
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (4 KB)
COMET INFO:     envi

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


In [7]:
for n in [8, 32]:
    perform_experiment(
        model=BaselineModel(SUSY.shape[1] - 1, n),
        X=SUSY.values[:,1:],
        y=SUSY.values[:,0],
        test_size=0.95,
        batch_size=2048,
        num_epochs=100,
        hyper_params={
            "model": "baseline",
            "dataset": "SUSY",
            "n": n,
        },
        log_every=10,
        random_state=42)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/9518b49165eb4152a178070fc8815861



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/9518b49165eb4152a178070fc8815861
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.8393643253064594, 0.8751830440004381)
COMET INFO:     train_batch_loss [1230] : (0.3424268662929535, 0.6925416588783264)
COMET INFO:     train_loss [1180]       : (0.3424268662929535, 0.694554328918457)
COMET INFO:     train_score [100]       : (0.839010703039893, 0.8759452555672219)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 8
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.95
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (4 KB)
COMET INFO:     envir

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/b59659dce9bf4294a0112f67ed142721



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/b59659dce9bf4294a0112f67ed142721
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.8625079351328111, 0.8756468786345084)
COMET INFO:     train_batch_loss [1230] : (0.37615084648132324, 0.6905171871185303)
COMET INFO:     train_loss [1180]       : (0.37615084648132324, 0.691317617893219)
COMET INFO:     train_score [100]       : (0.8617639822169381, 0.8806360743631283)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     model      : baseline
COMET INFO:     n          : 32
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.95
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (4 KB)
COMET INFO:     e

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


In [5]:
perform_experiment(
    model=BaselineModel(HIGGS.shape[1] - 1, 24),
    X=HIGGS.values[:,1:],
    y=HIGGS.values[:,0],
    test_size=0.99,
    batch_size=512,
    num_epochs=100,
    hyper_params={
        "model": "baseline",
        "dataset": "HIGGS",
        "n": 24,
    },
    log_every=2,
    random_state=42)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/841702b723984181ac6ae71b03c3d1fb



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/841702b723984181ac6ae71b03c3d1fb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]         : (0.6679174293543922, 0.7872175736072662)
COMET INFO:     train_batch_loss [10750] : (0.2768632471561432, 0.6995792388916016)
COMET INFO:     train_loss [2160]        : (0.2768632471561432, 0.6963033080101013)
COMET INFO:     train_score [100]        : (0.6688740320797226, 0.9358235776680406)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 512
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 24
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (3 KB)
COMET INFO:   

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


In [6]:
for test_size in [0.95]:
    for n in [8, 24]:
        perform_experiment(
            model=BaselineModel(HIGGS.shape[1] - 1, n),
            X=HIGGS.values[:,1:],
            y=HIGGS.values[:,0],
            test_size=test_size,
            batch_size=512,
            num_epochs=100,
            hyper_params={
                "model": "baseline",
                "dataset": "HIGGS",
                "n": n,
            },
            log_every=int((1 - test_size) * 200),
            random_state=42)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/f3fe2c65e2a145d8bd9ff59dc0859061



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/f3fe2c65e2a145d8bd9ff59dc0859061
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]         : (0.7197794788403695, 0.836219516064709)
COMET INFO:     train_batch_loss [10750] : (0.39437031745910645, 0.7062647342681885)
COMET INFO:     train_loss [10720]       : (0.39437031745910645, 0.7053879499435425)
COMET INFO:     train_score [100]        : (0.719347085846259, 0.8448127286359237)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 512
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 8
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.95
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (4 KB)
COMET INFO:    

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/05bbeace115b40acb2c58bad7506b734



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/05bbeace115b40acb2c58bad7506b734
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]         : (0.75121915395032, 0.8354711956244475)
COMET INFO:     train_batch_loss [10750] : (0.325880229473114, 0.6938849687576294)
COMET INFO:     train_loss [10720]       : (0.325880229473114, 0.6990153789520264)
COMET INFO:     train_score [100]        : (0.7513565698189232, 0.889915638688628)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 512
COMET INFO:     dataset    : HIGGS
COMET INFO:     model      : baseline
COMET INFO:     n          : 24
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.95
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (4 KB)
COMET INFO:     env

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


## ResNet model

In [10]:
class ResNetUnit(nn.Module):
    def __init__(self, external_size, internal_size):
        super().__init__()
        self.unit = nn.Sequential(
            nn.Linear(external_size, internal_size),
            nn.LeakyReLU(negative_slope=1e-1),
            nn.Linear(internal_size, external_size),
            nn.LeakyReLU(negative_slope=1e-1),
        )
        
    def forward(self, x):
        return x + self.unit(x)

In [13]:
class ResNetSequentialBlocks(nn.Module):
    def __init__(self, input_size, output_size, k_blocks, external_size, internal_size):
        super().__init__()
        self.blocks = nn.Sequential(*[
            nn.Linear(input_size, external_size),
            nn.LeakyReLU(negative_slope=1e-1),
        ] + [
            ResNetUnit(external_size, internal_size) for _ in range(k_blocks)
        ] + [
            nn.Linear(external_size, output_size),
        ])
    
    def forward(self, x):
        return self.blocks(x)

In [18]:
class ResNet(nn.Module):
    def __init__(self, input_size, n, k):
        super().__init__()
        self.model = nn.Sequential(
            ResNetSequentialBlocks(input_size, 8 * n, k, max(input_size, 8 * n), min(input_size, 8 * n)),
            nn.LeakyReLU(negative_slope=1e-1),
            ResNetSequentialBlocks(8 * n, 6 * n, k, 8 * n, 6 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            ResNetSequentialBlocks(6 * n, 4 * n, k, 6 * n, 4 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            ResNetSequentialBlocks(4 * n, 3 * n, k, 4 * n, 3 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            ResNetSequentialBlocks(3 * n, 2 * n, k, 3 * n, 2 * n),
            nn.LeakyReLU(negative_slope=1e-1),
            ResNetSequentialBlocks(2 * n, 1, k, 2 * n, 1),
            nn.Sigmoid()
        )
        
        def init_weights(layer):
            if type(layer) == nn.Linear:
                nn.init.normal_(layer.weight, std=1e-3)

        self.model.apply(init_weights)
    
    def forward(self, x):
        return self.model(x).reshape(-1)

In [ ]:
for test_size in [0.99, 0.9]:
    for n in [1, 4, 8, 16, 24, 32]:
        for k in [1, 4, 16]:
            perform_experiment(
                model=ResNet(SUSY.shape[1] - 1, n, k),
                X=SUSY.values[:,1:],
                y=SUSY.values[:,0],
                test_size=test_size,
                batch_size=512,
                num_epochs=100,
                hyper_params={
                    "model": "ResNet",
                    "dataset": "SUSY",
                    "n": n,
                    "k": k
                },
                log_every=int((1 - test_size) * 200),
                random_state=42)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/de6696caf74a471585186788b7a0d653
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [7]         : 0.5
COMET INFO:     train_batch_loss [100] : (0.6909967064857483, 0.7231723070144653)
COMET INFO:     train_loss [21]        : (0.6912254691123962, 0.7223334908485413)
COMET INFO:     train_score [8]        : (0.5, 0.8011566282784413)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 2048
COMET INFO:     dataset    : SUSY
COMET INFO:     k          : 1
COMET INFO:     model      : ResNet
COMET INFO:     n          : 1
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:  

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/52a525d5e0ee41e487938e152604fc1f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.5, 0.8714560755388525)
COMET INFO:     train_batch_loss [4900] : (0.3638288974761963, 0.779365062713623)
COMET INFO:     train_loss [980]        : (0.3638288974761963, 0.7804949879646301)
COMET INFO:     train_score [100]       : (0.5, 0.8711488640582667)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 512
COMET INFO:     dataset    : SUSY
COMET INFO:     k          : 1
COMET INFO:     model      : ResNet
COMET INFO:     n          : 1
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (12 KB)
COMET INFO:     envir

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/c0ef407e7e5b4209b2ab9dcbd7a2784c



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nikolai-semenov/resnet-tabular/c0ef407e7e5b4209b2ab9dcbd7a2784c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_score [100]        : (0.4999963523396863, 0.5000036476603138)
COMET INFO:     train_batch_loss [4900] : (0.676424503326416, 0.7024120092391968)
COMET INFO:     train_loss [980]        : (0.676424503326416, 0.7006546854972839)
COMET INFO:     train_score [100]       : (0.49998677396596164, 0.5000132260340384)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 512
COMET INFO:     dataset    : SUSY
COMET INFO:     k          : 4
COMET INFO:     model      : ResNet
COMET INFO:     n          : 1
COMET INFO:     num_epochs : 100
COMET INFO:     test_size  : 0.99
COMET INFO:   Uploads:
COMET INFO:     code                : 1

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nikolai-semenov/resnet-tabular/f12027f2b7114b5fa6a6309aacf1cdb1

